In [1]:
import numpy as np
#in case we need to repeat experiment
np.random.seed(255)

import pandas as pd
pd.options.display.max_rows = 22

import matplotlib.pyplot as plt
plt.style.use('classic')


import seaborn as sns
sns.set()

from sklearn.metrics import classification_report, confusion_matrix 

#Enable
enable_grid_search=False


In [2]:
#df = pd.read_csv('NCDB_2016.csv', engine = 'python')
df = pd.read_csv('data01_simple.csv', engine = 'python')
df

,C_YEAR,C_MNTH,C_WDAY,C_HOUR,C_VEHS,C_CONF,C_RCFG,C_WTHR,C_RSUR,C_RALN,...,V_ID,V_TYPE,V_YEAR,P_ID,P_SEX,P_AGE,P_PSN,P_SAFE,P_USER,P_ISEV
0,1999,1,1,9,2,34,2,1,1,1,...,1,1,1992,1,0,33,11,2,1,2
1,1999,1,1,9,2,34,2,1,1,1,...,2,1,1992,1,0,70,11,2,1,1
2,1999,1,1,20,1,3,3,1,1,3,...,1,1,1988,1,0,38,11,2,1,2
3,1999,1,1,5,2,1,2,3,2,4,...,1,6,1995,1,1,34,11,2,1,1
4,1999,1,1,5,2,1,2,3,2,4,...,1,6,1995,2,1,30,13,2,2,1
5,1999,1,1,8,3,33,2,4,2,4,...,1,1,1989,1,1,18,11,2,1,2
6,1999,1,1,9,2,1,2,1,1,1,...,1,1,1986,1,0,68,11,2,1,1
7,1999,1,1,14,2,21,2,1,2,1,...,2,1,1990,1,0,28,11,2,1,2
8,1999,1,1,14,2,21,2,1,2,1,...,2,1,1990,2,1,37,13,2,2,2
9,1999,1,1,7,1,4,5,1,5,1,...,1,6,1994,1,0,50,11,2,1,2


In [3]:
print(df.isnull().sum().sum())

0


In [4]:
print(df[df.index.astype('str').str.contains('[^0-9]')].sum().sum())

0


In [5]:
df_cat = df.astype('category').copy()

In [6]:
df_int = df.astype('int').copy()

In [7]:
print(df.columns)

Index(['C_YEAR', 'C_MNTH', 'C_WDAY', 'C_HOUR', 'C_VEHS', 'C_CONF', 'C_RCFG',
       'C_WTHR', 'C_RSUR', 'C_RALN', 'C_TRAF', 'V_ID', 'V_TYPE', 'V_YEAR',
       'P_ID', 'P_SEX', 'P_AGE', 'P_PSN', 'P_SAFE', 'P_USER', 'P_ISEV'],
      dtype='object')


## Convert Class Variable to Binary

In [8]:
## Convert Class Variable to Binary
### Merge Injury and Fatality as a single class
### we will compare the results.
df_binary_class = df_cat.copy()

#perform the conversion in two steps to avoid any unwanted side effects
df_binary_class['P_ISEV'] = df_binary_class['P_ISEV'].map({1: 'safe', 2: 'injury', 3:'fatal'})
df_binary_class['P_ISEV'] = df_binary_class['P_ISEV'].map({'safe': '0', 'injury': '1', 'fatal':'1'})
print((df_binary_class['P_ISEV']=='0').sum())
print((df_binary_class['P_ISEV']=='1').sum())
print(df_binary_class['P_ISEV'].unique())

1570775
2084559
['1' '0']


## Split Training and Testing for Binary class

In [9]:
#Split between data and class
Ybinary = df_binary_class[df_binary_class.columns[-1]]
Xbinary = df_binary_class[df_binary_class.columns[0:df_binary_class.columns.size -1]]
#print(Xbinary, Ybinary)

#### Split Test(70%) and Train (30%) for Bianry class 

In [10]:
#sprint into train and test 70/30
#from sklearn.cross_validation import train_test_split
from sklearn.model_selection import train_test_split
Xbinary_train, Xbinary_test, Ybinary_train, Ybinary_test = train_test_split(Xbinary, Ybinary, test_size=0.3, random_state=0)

In [11]:
print(Xbinary_train, Xbinary_test, Ybinary_train, Ybinary_test)

        C_YEAR C_MNTH C_WDAY C_HOUR C_VEHS C_CONF C_RCFG C_WTHR C_RSUR C_RALN  \
1890237   2007      5      7     22      2     21      2      3      2      3   
2678999   2011      7      5     11      2     21      1      1      1      1   
65248     1999      5      1     17      2     32      1      3      2      3   
3081880   2013      9      7     19      1      4      1      1      1      1   
262828    2000      2      7     15      2     21      2      6      2      1   
91440     1999      6      3     15      1      4      1      2      1      2   
424903    2000     11      2     12      2     21      1      3      2      1   
962859    2003      2      4     21      2     21      1      1      1      1   
2635432   2011      4      6      8      2      6      1      3      2      1   
1424524   2005      3      4     15      1      6      1      4      2      3   
517868    2001      4      2     21      2     41      1      1      1      1   
...        ...    ...    ...

## Undersample the majority for the 3 class evaluation

#### Subset Data into class Fatal

In [12]:
# fatal
is_fatal =  df_cat['P_ISEV']==3
is_fatal_count = is_fatal.sum()
print(is_fatal_count)
df_cat_fatal = df_cat[is_fatal]
display(df_cat_fatal)


23081


,C_YEAR,C_MNTH,C_WDAY,C_HOUR,C_VEHS,C_CONF,C_RCFG,C_WTHR,C_RSUR,C_RALN,...,V_ID,V_TYPE,V_YEAR,P_ID,P_SEX,P_AGE,P_PSN,P_SAFE,P_USER,P_ISEV
75,1999,1,1,22,1,2,1,1,1,1,...,1,1,1991,1,1,74,11,2,1,3
213,1999,1,1,20,2,35,2,3,2,1,...,1,6,1990,1,1,60,11,13,1,3
291,1999,1,1,11,2,31,1,1,5,4,...,2,1,1988,1,1,30,11,2,1,3
292,1999,1,1,11,2,31,1,1,5,4,...,2,1,1988,2,0,27,13,2,2,3
293,1999,1,1,11,2,31,1,1,5,4,...,2,1,1988,3,1,7,23,2,2,3
417,1999,1,1,10,2,21,2,4,3,2,...,1,1,1993,2,0,58,13,2,2,3
419,1999,1,1,12,2,1,5,1,2,1,...,1,1,1991,1,1,42,11,2,1,3
424,1999,1,1,22,2,31,1,4,3,1,...,2,1,1986,2,0,54,13,2,2,3
425,1999,1,1,16,3,31,1,1,1,1,...,1,1,1998,1,0,71,11,2,1,3
427,1999,1,1,16,3,31,1,1,1,1,...,2,1,1992,2,0,84,13,2,2,3


#### Subset Data into class Injury

In [13]:
# injury
is_injury =  df_cat['P_ISEV']==2
is_injury_count = is_injury.sum()
print(is_injury_count)
df_cat_injury = df_cat[is_injury]
display(df_cat_injury)


2061478


,C_YEAR,C_MNTH,C_WDAY,C_HOUR,C_VEHS,C_CONF,C_RCFG,C_WTHR,C_RSUR,C_RALN,...,V_ID,V_TYPE,V_YEAR,P_ID,P_SEX,P_AGE,P_PSN,P_SAFE,P_USER,P_ISEV
0,1999,1,1,9,2,34,2,1,1,1,...,1,1,1992,1,0,33,11,2,1,2
2,1999,1,1,20,1,3,3,1,1,3,...,1,1,1988,1,0,38,11,2,1,2
5,1999,1,1,8,3,33,2,4,2,4,...,1,1,1989,1,1,18,11,2,1,2
7,1999,1,1,14,2,21,2,1,2,1,...,2,1,1990,1,0,28,11,2,1,2
8,1999,1,1,14,2,21,2,1,2,1,...,2,1,1990,2,1,37,13,2,2,2
9,1999,1,1,7,1,4,5,1,5,1,...,1,6,1994,1,0,50,11,2,1,2
10,1999,1,1,13,2,24,3,1,1,1,...,1,1,1992,1,1,20,11,1,1,2
11,1999,1,1,8,2,34,2,1,2,1,...,1,6,1989,1,1,53,11,2,1,2
12,1999,1,1,8,2,34,2,1,2,1,...,2,1,1998,1,0,71,11,2,1,2
13,1999,1,1,15,2,21,2,1,1,1,...,2,1,1990,2,1,26,23,2,2,2


#### Subset Data into class safe

In [14]:
# non_injury
is_safe =  df_cat['P_ISEV']==1
#print(is_safe)
is_safe_count = is_safe.sum()
print(is_safe_count)
df_cat_safe = df_cat[is_safe]
display(df_cat_safe)

1570775


,C_YEAR,C_MNTH,C_WDAY,C_HOUR,C_VEHS,C_CONF,C_RCFG,C_WTHR,C_RSUR,C_RALN,...,V_ID,V_TYPE,V_YEAR,P_ID,P_SEX,P_AGE,P_PSN,P_SAFE,P_USER,P_ISEV
1,1999,1,1,9,2,34,2,1,1,1,...,2,1,1992,1,0,70,11,2,1,1
3,1999,1,1,5,2,1,2,3,2,4,...,1,6,1995,1,1,34,11,2,1,1
4,1999,1,1,5,2,1,2,3,2,4,...,1,6,1995,2,1,30,13,2,2,1
6,1999,1,1,9,2,1,2,1,1,1,...,1,1,1986,1,0,68,11,2,1,1
39,1999,1,1,9,2,35,2,1,3,1,...,1,1,1995,1,0,44,11,2,1,1
42,1999,1,1,13,1,1,1,2,3,1,...,1,1,1988,1,1,18,11,2,1,1
44,1999,1,1,5,2,35,1,1,2,1,...,2,8,1999,1,1,37,11,2,1,1
53,1999,1,1,15,1,1,2,2,2,1,...,1,1,1990,1,1,55,11,2,1,1
60,1999,1,1,13,1,1,1,3,2,1,...,1,6,1998,1,1,41,11,2,1,1
62,1999,1,1,14,3,21,1,1,2,2,...,1,1,1991,1,0,29,11,2,1,1


In [15]:
# get the size of fatal datafram
min_size = df_cat_fatal.index.size
print(min_size)

23081


In [16]:
# get size of injury
df_cat_safe.index.size

1570775

In [17]:
# size of injury
df_cat_injury.index.size

2061478

## Create Final dataframe for tenary class modeling

In [18]:
# randomly sample n number of injury and no injury and append to fatal
df_cat_injury_select = df_cat_injury.sample(n=min_size)
df_cat_safe_select = df_cat_safe.sample(n=min_size)


In [19]:
df_cat_injury_select.shape

(23081, 21)

In [20]:
df_cat_safe_select.shape

(23081, 21)

In [21]:
#concat the three dataframes
df_final = pd.concat([df_cat_fatal, df_cat_injury_select, df_cat_safe_select])
df_final.shape

(69243, 21)

In [22]:
df_final.P_ISEV.unique()

[3, 2, 1]
Categories (3, int64): [3, 2, 1]

## Write cleaned data to file for future use.

In [23]:
#lets write the datafile for future use
df_final.to_csv('cleandata.csv', encoding='utf-8', index=False)

In [24]:
#Split between data and class

Y = df_final[df_cat.columns[-1]]
X = df_final[df_cat.columns[0:df_cat.columns.size -1]]
#print(X,Y)

        C_YEAR C_MNTH C_WDAY C_HOUR C_VEHS C_CONF C_RCFG C_WTHR C_RSUR C_RALN  \
75        1999      1      1     22      1      2      1      1      1      1   
213       1999      1      1     20      2     35      2      3      2      1   
291       1999      1      1     11      2     31      1      1      5      4   
292       1999      1      1     11      2     31      1      1      5      4   
293       1999      1      1     11      2     31      1      1      5      4   
417       1999      1      1     10      2     21      2      4      3      2   
419       1999      1      1     12      2      1      5      1      2      1   
424       1999      1      1     22      2     31      1      4      3      1   
425       1999      1      1     16      3     31      1      1      1      1   
427       1999      1      1     16      3     31      1      1      1      1   
429       1999      1      1     17      1      1      1      4      3      1   
...        ...    ...    ...

## Clustering based on K-Means Clustering

In [25]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=3, init='random', n_init=10, max_iter=300, tol=1e-04)
ykm = kmeans.fit(X)

In [26]:
ykm.cluster_centers_

array([[2.00732926e+03, 6.79474834e+00, 3.95941900e+00, 1.36219453e+01,
        2.12781194e+00, 2.51346092e+01, 1.65742839e+00, 1.58933484e+00,
        1.55300210e+00, 1.49223175e+00, 1.13332659e+01, 1.53281275e+00,
        2.01873280e+00, 1.99976331e+03, 1.29964396e+00, 5.84236932e-01,
        5.99805389e+01, 1.23080596e+01, 2.35470950e+00, 1.37489885e+00],
       [2.00653847e+03, 6.70038620e+00, 4.09947100e+00, 1.38332522e+01,
        2.31769059e+00, 2.85592445e+01, 1.72381787e+00, 1.61272839e+00,
        1.55119592e+00, 1.39220459e+00, 1.05284133e+01, 1.59870834e+00,
        1.79975984e+00, 1.99883546e+03, 1.62301626e+00, 5.65280888e-01,
        2.53035083e+01, 1.38984195e+01, 2.34618505e+00, 1.48972836e+00],
       [2.00643087e+03, 6.87479947e+00, 4.48527053e+00, 1.18049439e+01,
        1.08859560e+00, 4.30377716e+00, 1.33243401e+00, 1.70132711e+00,
        1.77293277e+00, 2.02625055e+00, 1.64145399e+01, 1.03346945e+00,
        2.38165378e+00, 1.99813803e+03, 1.54586554e+00, 6.8105

In [27]:
display(ykm.labels_)

array([0, 0, 1, ..., 2, 1, 1])

## Feature selection using Random Forest

### Feature Selection 

In [28]:
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(n_estimators=1000, random_state=0, n_jobs=-1)
forest.fit(X, Y)

C:\Users\pradeep\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=-1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

### Get the inportant features from random forest

In [29]:
importFeatures = forest.feature_importances_

### List the features by importancce

In [30]:
importFeatures

array([0.08161322, 0.07424822, 0.06118096, 0.09192552, 0.02999759,
       0.10265134, 0.03308105, 0.02775738, 0.02745039, 0.03186772,
       0.03506826, 0.02518782, 0.02093322, 0.09368957, 0.01703348,
       0.01923754, 0.12259927, 0.01941274, 0.06838534, 0.01667936])

In [31]:
indices = np.argsort(importFeatures)[::-1]
print(indices)
featureLabel = X.columns[0:]
print(featureLabel)
rankedFeature = []
for f in range(X.shape[1]):
    rankedFeature.append(featureLabel[indices[f]])
    print("%2d) %-*s %f" % (f+1, 30,  featureLabel[indices[f]], importFeatures[indices[f]]))
print(rankedFeature)

[16  5 13  3  0  1 18  2 10  6  9  4  7  8 11 12 17 15 14 19]
Index(['C_YEAR', 'C_MNTH', 'C_WDAY', 'C_HOUR', 'C_VEHS', 'C_CONF', 'C_RCFG',
       'C_WTHR', 'C_RSUR', 'C_RALN', 'C_TRAF', 'V_ID', 'V_TYPE', 'V_YEAR',
       'P_ID', 'P_SEX', 'P_AGE', 'P_PSN', 'P_SAFE', 'P_USER'],
      dtype='object')
 1) P_AGE                          0.122599
 2) C_CONF                         0.102651
 3) V_YEAR                         0.093690
 4) C_HOUR                         0.091926
 5) C_YEAR                         0.081613
 6) C_MNTH                         0.074248
 7) P_SAFE                         0.068385
 8) C_WDAY                         0.061181
 9) C_TRAF                         0.035068
10) C_RCFG                         0.033081
11) C_RALN                         0.031868
12) C_VEHS                         0.029998
13) C_WTHR                         0.027757
14) C_RSUR                         0.027450
15) V_ID                           0.025188
16) V_TYPE                         0.0209

### Reduce the number of features

In [32]:
#select features that contribute more than 0.05
#[df_cat.columns[0:df_cat.columns.size -1]]
X_Selected = X[rankedFeature[0:10]]
display(X_Selected)
X_Selected.shape

#sprint into train and test 70/30
#from sklearn.cross_validation import train_test_split
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X_Selected, Y, test_size=0.3, random_state=0)

,P_AGE,C_CONF,V_YEAR,C_HOUR,C_YEAR,C_MNTH,P_SAFE,C_WDAY,C_TRAF,C_RCFG
75,74,2,1991,22,1999,1,2,1,18,1
213,60,35,1990,20,1999,1,13,1,1,2
291,30,31,1988,11,1999,1,2,1,18,1
292,27,31,1988,11,1999,1,2,1,18,1
293,7,31,1988,11,1999,1,2,1,18,1
417,58,21,1993,10,1999,1,2,1,18,2
419,42,1,1991,12,1999,1,2,1,18,5
424,54,31,1986,22,1999,1,2,1,18,1
425,71,31,1998,16,1999,1,2,1,18,1
427,84,31,1992,16,1999,1,2,1,18,1


### SVM GridSearch for Optimal Parms

In [33]:
#This operation is computationaly expensive.
#Enable as required.
enable_grid_search = False
if enable_grid_search:
    from sklearn.grid_search import GridSearchCV
    from sklearn.svm import SVC
    param_grid = {'C':[0.1, 1, 10, 100, 1000], 'gamma':[1, 0.1, 0.01, 0.001, 0.0001]}
    grid = GridSearchCV(SVC(), param_grid, verbose=3)
    grid.fit(X_train, Y_train)
    print(grid.best_params_)
    #{'C': 1000, 'gamma': 0.001}
    print(grid.best_estimator_)
    #SVC(C=1000, cache_size=200, class_weight=None, coef0=0.0,
    #  decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
    #  max_iter=-1, probability=False, random_state=None, shrinking=True,
    #  tol=0.001, verbose=False)
    grid_predictions = grid.predict(X_test)
    from sklearn.metrics import confusion_matrix
    confusion_matrix = confusion_matrix(Y_test, grid_predictions)
    print(confusion_matrix)
    print(classification_report(Y_test,grid_predictions))

C:\Users\pradeep\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\pradeep\Anaconda3\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


## Logistic Regression Model

In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(C=1, random_state=0)
lr.fit(X_train, Y_train)

In [ ]:
y_pred = lr.predict(X_test)
#display(y_pred)
print('Accuracy of logistic regression classifier on train set: {:.2f}'.format(lr.score(X_train, Y_train)))
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(lr.score(X_test, Y_test)))

In [ ]:
# print the intercept (Note: one vs rest => 1 vs 2and3, 2 vs 1and3, 3 vs 1and2)
lr.intercept_

In [ ]:
# print the coeficients (Note: one vs rest => 1 vs 2and3, 2 vs 1and3, 3 vs 1and2)
lr.coef_

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(Y_test, y_pred)
print(confusion_matrix)
print(classification_report(Y_test,y_pred))

In [ ]:
# with L1 regularization
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(penalty='l1', C=1000, random_state=0)
lr.fit(X_train, Y_train)

In [ ]:
y_pred = lr.predict(X_test)
print('Accuracy of logistic regression classifier on train set: {:.2f}'.format(lr.score(X_train, Y_train)))
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(lr.score(X_test, Y_test)))

from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(Y_test, y_pred)
print(confusion_matrix)
print(classification_report(Y_test,y_pred))

### Support Vector Machines

In [ ]:
from sklearn.svm import SVC
svm = SVC(kernel='sigmoid', C=1, random_state=0)
svm.fit(X_train, Y_train)

In [ ]:
y_pred = svm.predict(X_test)
print('Accuracy of logistic regression classifier on train set: {:.2f}'.format(lr.score(X_train, Y_train)))
print('Accuracy of SVM regression classifier on test set: {:.2f}'.format(svm.score(X_test, Y_test)))

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(Y_test, y_pred)
print(confusion_matrix)
print(classification_report(Y_test,y_pred))

### Decison Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier(criterion='entropy',max_depth=5, random_state=0)
tree.fit(X_train, Y_train)

In [ ]:
y_pred = tree.predict(X_test)
print('Accuracy of logistic regression classifier on train set: {:.2f}'.format(lr.score(X_train, Y_train)))
print('Accuracy of Tree classifier on test set: {:.2f}'.format(tree.score(X_test, Y_test)))

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(Y_test, y_pred)
print(confusion_matrix)
print(classification_report(Y_test,y_pred))

### Random forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(criterion='entropy', n_estimators=1000, random_state=0, n_jobs=2)
forest.fit(X_train, Y_train)

In [ ]:
y_pred = forest.predict(X_test)
print('Accuracy of logistic regression classifier on train set: {:.2f}'.format(lr.score(X_train, Y_train)))
print('Accuracy of RandomForest classifier on test set: {:.2f}'.format(forest.score(X_test, Y_test)))

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(Y_test, y_pred)
print(confusion_matrix)
print(classification_report(Y_test,y_pred))

### K-N-N

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5, p=2, metric='minkowski')
knn.fit(X_train, Y_train)

In [ ]:
y_pred = knn.predict(X_test)
print('Accuracy of logistic regression classifier on train set: {:.2f}'.format(lr.score(X_train, Y_train)))
print('Accuracy of RandomForest classifier on test set: {:.2f}'.format(forest.score(X_test, Y_test)))

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(Y_test, y_pred)
print(confusion_matrix)
print(classification_report(Y_test,y_pred))

### SVM

In [ ]:
from sklearn.svm import SVC
#svm = SVC(C=1, random_state=0, kernel='sigmoid', verbose=True)
#svm = SVC(C=1, random_state=0, kernel='linear', verbose=True, cache_size=200)
svm = SVC(verbose = 3)

In [ ]:
svm.fit(X_train, Y_train)

In [ ]:
y_pred = svm.predict(X_test)
print('Accuracy of logistic regression classifier on train set: {:.2f}'.format(lr.score(X_train, Y_train)))
print('Accuracy of SVM regression classifier on test set: {:.2f}'.format(svm.score(X_test, Y_test)))

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(Y_test, y_pred)
print(confusion_matrix)
print(classification_report(Y_test,y_pred))

### Performance Tuning using GridSearch

In [ ]:
from sklearn.grid_search import GridSearchCV
from sklearn.svm import SVC
param_grid = {'C':[0.1, 1, 10, 100, 1000], 'gamma':[1, 0.1, 0.01, 0.001, 0.0001]}
grid = GridSearchCV(SVC(), param_grid, verbose=3)
grid.fit(X_train, Y_train)

In [ ]:
grid.best_parms_

In [ ]:
grid.best_estimator

In [ ]:
grid_predictions = grid.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(Y_test, grid_predictions)
print(confusion_matrix)
print(classification_report(Y_test,grid_predictions))

In [ ]:

#check sigmoid and rbf
#from sklearn.ensemble import BaggingClassifier
#from sklearn.svm import SVC
#clf = BaggingClassifier(SVC(C=1.0,
#        cache_size=200,
#        class_weight=None,
#        coef0=0.0,
#        decision_function_shape=None,
#        degree=3,
#        gamma='auto',
#        kernel='linear',
#        max_iter=-1,
#        probability=False,
#        random_state=None,
#        shrinking=True,
#        tol=0.001,
#        verbose=False,
#        ))

In [ ]:
#clf.fit(X_train, Y_train)